In [1]:
import cvxpy as cp
import numpy as np

In [2]:
number_horizontal_spaces = 4
number_vertical_spaces = 2

In [3]:
def validate_input(names, values, targeted_weights, number_payments, payment):
    matching_number_of_inputs = len(names) == len(values) and len(values) == len(targeted_weights)
    if not matching_number_of_inputs:
        print("Error: Es müssen gleich viele Namen, aktuelle Werte und Gewichte eingegeben werden!")
    
    weights_sum_up_to_1 = sum(targeted_weights) == 1
    if not weights_sum_up_to_1:
        print("Error: Die gewünschten Gewichte müssen sich zu 100% aufsummieren!")
    
    number_payments_is_bigger_1 = number_payments > 0
    if not number_payments_is_bigger_1:
        print("Error: Die Anzahl der Einzahlungen muss eine ganze Zahl größer gleich 1 sein!")
    
    payment_is_positive = payment >= 0
    if not payment_is_positive:
        print("Error: Die Einzahlung muss positiv sein. Dieser Rechner funktioniert nicht für Auszahlungen!")
    
    return matching_number_of_inputs and weights_sum_up_to_1 and number_payments_is_bigger_1 and payment_is_positive

In [4]:
# min |weights*(sum(values)+npay*pay)-(values+x*npay*pay)|^2 s.t. x>=0, sum(x)==1
# <=> min |(weights*(sum(values)+npay*pay)-values)/(npay*pay)-x|^2 s.t. x>=0, sum(x)==1
def calculate_distribution(values, targeted_weights, number_payments, payment):
    n = targeted_weights.size
    total_amount = values.sum() + number_payments * payment
    x = cp.Variable(n)
    c = (targeted_weights * total_amount - values) / (number_payments * payment)
    prob = cp.Problem(
        cp.Minimize(cp.quad_form(x, np.eye(n)) - 2 * c.T @ x + c.T @ c),
        [x >= 0, np.ones(n) @ x == 1]
    )
    difference = prob.solve()
    distribution = np.round(x.value * payment)
    distribution = np.where(distribution>=0, distribution, 0)
    return distribution, difference

In [5]:
def print_statistics(names, values, targeted_weights, number_payments, distribution, difference):
    n = len(names)
    current_values = values.round(2)
    current_percentages = np.round(100 * current_values / current_values.sum(), 1)
    targeted_percentages = np.round(targeted_weights * 100, 1)
    payment = distribution.sum()
    new_values = values + number_payments * distribution
    new_percentages = np.round(100 * new_values / new_values.sum(), 1)
    
    names_max_length = max(max(map(len, names)), 4)
    current_values_max_length = max(max(map(len, map(str, current_values))), 14)
    current_percentages_max_length = max(1+max(map(len, map(str, current_percentages))), 17)
    new_values_max_length = max(max(map(len, map(str, new_values))), 10)
    new_percentages_max_length = max(1+max(map(len, map(str, new_percentages))), 13)
    targeted_percentages_max_length = max(1+max(map(len, map(str, targeted_weights))), 20)
    
    print(int(number_vertical_spaces/2)*"\n")
    print((names_max_length + current_values_max_length + current_percentages_max_length + new_values_max_length + \
           new_percentages_max_length + targeted_percentages_max_length) * "-")
    print(int(number_vertical_spaces/2)*"\n")
    
    print(f"Zahle die nächsten {number_payments} mal jeweils folgendes ein:")
    print("Name", (names_max_length-4+number_horizontal_spaces)*" ", "Einzahlung")
    for name, dist in zip(names, distribution):
        print(name, (names_max_length-len(name)+number_horizontal_spaces)*" ", dist)
    print(number_vertical_spaces*"\n")
    
    print("Damit verändert sich das Portfolio folgendermaßen:")
    print("Name", (names_max_length-4+number_horizontal_spaces)*" ",
          "aktueller Wert", (current_values_max_length-14+number_horizontal_spaces)*" ",
          "aktuelles Gewicht", (current_percentages_max_length-17+number_horizontal_spaces)*" ",
          6*" ",
          "neuer Wert", (new_values_max_length-10+number_horizontal_spaces)*" ",
          "neues Gewicht", (new_percentages_max_length-13+number_horizontal_spaces)*" ",
          "gewünschtes Gewicht", (targeted_percentages_max_length-19+number_horizontal_spaces)*" "
    )
    for name, current_value, current_percentage, new_value, new_percentage, targeted_percentage in zip(names, current_values, current_percentages, new_values, new_percentages, targeted_percentages):
        print(name, (names_max_length-len(name)+number_horizontal_spaces)*" ",
              current_value, (current_values_max_length-len(str(current_value))+number_horizontal_spaces)*" ",
              current_percentage, (current_percentages_max_length-len(str(current_percentage))+number_horizontal_spaces)*" ",
              2*" " + "=>" + 2*" ",
              new_value, (new_values_max_length-len(str(new_value))+number_horizontal_spaces)*" ",
              new_percentage, (new_percentages_max_length-len(str(new_percentage))+number_horizontal_spaces)*" ",
              targeted_percentage, (targeted_percentages_max_length-len(str(targeted_percentage))+number_horizontal_spaces)*" "
        )
    print()
    
    if difference < 1e-3:
        print("Die gewünschten Gewichte lassen sich erreichen!")
    else:
        print("Die gewünschten Gewichte lassen sich nicht erreichen. Kein Grund zur Panik!",
              "Können von den ETFs die ein zu großes Gewicht haben, vielleicht Teile steuereffizient verkauft werden?")

In [6]:
names = input("Bitte gebe die Namen der ETFs mit Kommas getrennt ein.\nBsp.: World, EM IMI, USA SCV, Europe SCV\n")
names = list(map(lambda s: s.strip(), names.split(",")))
print()

values = input("Bitte gebe die aktuellen Werte der ETFs mit Kommas getrennt ein.\nKommazahlen bitte mit Punkt statt Komma eingeben.\nBitte die gleiche Reihenfolge wie bei den Namen verwenden.\nBsp.: 6500, 1100, 1400, 1000\n")
values = np.array(list(map(lambda v: float(v), values.split(","))))
print()

targeted_weights = input("Bitte gebe die gewünschten Gewichte in Prozent der ETFs mit Kommas getrennt ein.\nKommazahlen bitte mit Punkt statt Komma eingeben.\nBitte die gleiche Reihenfolge wie bei den Namen verwenden.\nBsp.: 65, 11, 14, 10\n")
targeted_weights = np.array(list(map(lambda w: float(w), targeted_weights.split(",")))) / 100
print()

number_payments = input("Bitte gebe bei einer Einmaleinzahlung eine 1 ein.\nBei einem Sparplan bitte die Anzahl an Einzahlungen eingeben bei denen der Sparplan nicht verändert wird:\n")
number_payments = int(number_payments)
print()

payment = input("Bitte gebe die Höhe der Einzahlung an. Bei einem Sparplan nur der Betrag für eine Ausführung:\n")
payment = float(payment)

input_is_valid = validate_input(names, values, targeted_weights, number_payments, payment)
if input_is_valid:
    distribution, difference = calculate_distribution(values, targeted_weights, number_payments, payment)
    print_statistics(names, values, targeted_weights, number_payments, distribution, difference)







------------------------------------------------------------------------------------


Zahle die nächsten 1 mal jeweils folgendes ein:
Name            Einzahlung
World           3250.0
EM IMI          550.0
USA SCV         700.0
Europe SCV      500.0



Damit verändert sich das Portfolio folgendermaßen:
Name            aktueller Wert      aktuelles Gewicht             neuer Wert      neues Gewicht      gewünschtes Gewicht      
World           6500.0              65.0                     =>   9750.0          65.0               65.0                     
EM IMI          1100.0              11.0                     =>   1650.0          11.0               11.0                     
USA SCV         1400.0              14.0                     =>   2100.0          14.0               14.0                     
Europe SCV      1000.0              10.0                     =>   1500.0          10.0               10.0                     

Die gewünschten Gewichte lassen sich erreichen!
